 # Capstone - The Battle of Neighborhoods

## Import Libraries

Importing the required libraries

In [26]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium

import urllib.request
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



## Download data


Dowload Neighborhood data with 5 boroughs and 306 neighborhoods (including latitude and longitude).
Dataset documentation: https://geo.nyu.edu/catalog/nyu_2451_34572
Dataset location: http://cocl.us/new_york_dataset/

In [27]:
url = "http://cocl.us/new_york_dataset/"
json_url = urlopen(url)
NY_data = json.loads(json_url.read())

#### Creating Dataframe

In [30]:
neighborhoods_data = NY_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


#### Getting latitude and longitude information with geopy.

In [31]:
add = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(add)
longitude = location.longitude
latitude = location.latitude

#### Creating a map with the data

In [33]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


In [34]:
import folium
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
map_newyork

## Foursquare venues


In [36]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [37]:
#Foursquare parameters
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [38]:
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d111941735')
newyork_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Empire Chinese And Japanese Cuisine,40.878588,-73.917446,Japanese Restaurant
1,Marble Hill,40.876551,-73.910660,Sushikawa Restaurant,40.887341,-73.906967,Japanese Restaurant
2,Marble Hill,40.876551,-73.910660,Planet Tokyo,40.886233,-73.909479,Sushi Restaurant
3,Marble Hill,40.876551,-73.910660,Palace of Japan,40.885841,-73.910064,Japanese Restaurant
4,Marble Hill,40.876551,-73.910660,Aoyu Japanese Restaurant,40.886257,-73.909719,Japanese Restaurant
5,Chinatown,40.715618,-73.994279,Shinka Ramen & Sake Bar,40.717147,-73.995206,Japanese Restaurant
6,Chinatown,40.715618,-73.994279,Tofu Tofu,40.717605,-73.995304,Korean Restaurant
7,Chinatown,40.715618,-73.994279,Kissaki,40.725372,-73.991847,Sushi Restaurant
8,Chinatown,40.715618,-73.994279,Shōji,40.717582,-74.005592,Sushi Restaurant
9,Chinatown,40.715618,-73.994279,Kintsugi,40.722969,-74.004620,Sushi Restaurant


In [39]:
newyork_venues.shape

(1613, 7)

In [23]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [40]:
#def addColumn(startDf, columnTitle, dataDf):
#    grouped = dataDf.groupby('Neighborhood').count()    
#    for n in startDf['Neighborhood']:
#        try:
#            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
#        except:
#            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [42]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues, 'blue', map_newyork)
map_newyork

In [43]:
manhattan_grouped = newyork_venues.groupby('Neighborhood').count()
manhattan_grouped.head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,50,50,50,50,50,50
Carnegie Hill,42,42,42,42,42,42
Central Harlem,21,21,21,21,21,21
Chelsea,50,50,50,50,50,50
Chinatown,50,50,50,50,50,50
Civic Center,49,49,49,49,49,49
Clinton,50,50,50,50,50,50
East Harlem,5,5,5,5,5,5
East Village,50,50,50,50,50,50


## 3. Neighborhood Analysis

In [44]:
manhattan_onehot = pd.get_dummies(newyork_venues_sushi[['Venue Category']], prefix="", prefix_sep="")
manhattan_onehot['Neighborhood'] = newyork_venues_sushi['Neighborhood'] 
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]
manhattan_onehot.head(10)

,Neighborhood,American Restaurant,Asian Restaurant,Beer Bar,Bubble Tea Shop,Building,Café,Chinese Restaurant,Cocktail Bar,Coffee Shop,Deli / Bodega,Dessert Shop,Fish Market,Food & Drink Shop,Food Truck,French Restaurant,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Korean Restaurant,New American Restaurant,Noodle House,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant,Shabu-Shabu Restaurant,Smoothie Shop,Soba Restaurant,Sushi Restaurant,Taco Place,Tea Room,Tonkatsu Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
8,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head(10)

,Neighborhood,American Restaurant,Asian Restaurant,Beer Bar,Bubble Tea Shop,Building,Café,Chinese Restaurant,Cocktail Bar,Coffee Shop,Deli / Bodega,Dessert Shop,Fish Market,Food & Drink Shop,Food Truck,French Restaurant,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Korean Restaurant,New American Restaurant,Noodle House,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant,Shabu-Shabu Restaurant,Smoothie Shop,Soba Restaurant,Sushi Restaurant,Taco Place,Tea Room,Tonkatsu Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.00,0.020000,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.02,0.0,0.0,0.0,0.00000,0.020000,0.400000,0.000000,0.0,0.020000,0.080000,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.440000,0.0,0.00,0.0,0.00,0.00000
1,Carnegie Hill,0.00,0.047619,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.02381,0.000000,0.285714,0.000000,0.0,0.000000,0.047619,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.571429,0.0,0.00,0.0,0.00,0.02381
2,Central Harlem,0.00,0.000000,0.0,0.0,0.0,0.00,0.00,0.047619,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00000,0.095238,0.333333,0.000000,0.0,0.047619,0.095238,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.380952,0.0,0.00,0.0,0.00,0.00000
3,Chelsea,0.00,0.020000,0.0,0.0,0.0,0.00,0.00,0.020000,0.00,0.02,0.00,0.02,0.0,0.00,0.0,0.0,0.0,0.00000,0.020000,0.340000,0.000000,0.0,0.000000,0.120000,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.420000,0.0,0.00,0.0,0.02,0.00000
4,Chinatown,0.00,0.020000,0.0,0.0,0.0,0.02,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00000,0.000000,0.460000,0.040000,0.0,0.040000,0.120000,0.00,0.02,0.0,0.00,0.00,0.00,0.00,0.280000,0.0,0.00,0.0,0.00,0.00000
5,Civic Center,0.00,0.020408,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00000,0.040816,0.346939,0.020408,0.0,0.061224,0.081633,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.428571,0.0,0.00,0.0,0.00,0.00000
6,Clinton,0.02,0.020000,0.0,0.0,0.0,0.00,0.02,0.020000,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00000,0.040000,0.260000,0.000000,0.0,0.000000,0.280000,0.02,0.00,0.0,0.02,0.00,0.02,0.00,0.280000,0.0,0.00,0.0,0.00,0.00000
7,East Harlem,0.00,0.000000,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.2,0.00000,0.000000,0.200000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.600000,0.0,0.00,0.0,0.00,0.00000
8,East Village,0.00,0.020000,0.0,0.0,0.0,0.02,0.00,0.000000,0.02,0.00,0.02,0.00,0.0,0.00,0.0,0.0,0.0,0.00000,0.020000,0.380000,0.000000,0.0,0.000000,0.140000,0.00,0.04,0.0,0.00,0.02,0.00,0.02,0.260000,0.0,0.02,0.0,0.02,0.00000
9,Financial District,0.00,0.020000,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.02,0.0,0.0,0.0,0.00000,0.020000,0.420000,0.000000,0.0,0.000000,0.060000,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.460000,0.0,0.00,0.0,0.00,0.00000


In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']
for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Japanese Curry Restaurant,Asian Restaurant,Food Truck,Noodle House,Deli / Bodega,Food & Drink Shop,Fish Market
1,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Building,Café,Chinese Restaurant,Cocktail Bar
2,Central Harlem,Sushi Restaurant,Japanese Restaurant,Japanese Curry Restaurant,Ramen Restaurant,Noodle House,Cocktail Bar,Deli / Bodega,Food Truck,Food & Drink Shop,Fish Market
3,Chelsea,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Japanese Curry Restaurant,Asian Restaurant,Udon Restaurant,Fish Market,Cocktail Bar,Deli / Bodega,Dessert Shop
4,Chinatown,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Korean Restaurant,Noodle House,Asian Restaurant,Café,Sake Bar,Vegetarian / Vegan Restaurant,Food & Drink Shop
5,Civic Center,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Noodle House,Japanese Curry Restaurant,Asian Restaurant,Korean Restaurant,Dessert Shop,Food Truck,Food & Drink Shop
6,Clinton,Sushi Restaurant,Ramen Restaurant,Japanese Restaurant,Japanese Curry Restaurant,Seafood Restaurant,Asian Restaurant,Chinese Restaurant,Cocktail Bar,Restaurant,American Restaurant
7,East Harlem,Sushi Restaurant,Japanese Restaurant,Hawaiian Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,French Restaurant,Food Truck,Food & Drink Shop,Fish Market,Dessert Shop
8,East Village,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Sake Bar,Café,Dessert Shop,Udon Restaurant,Coffee Shop,Japanese Curry Restaurant,Shabu-Shabu Restaurant
9,Financial District,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Japanese Curry Restaurant,Asian Restaurant,Food Truck,Deli / Bodega,French Restaurant,Food & Drink Shop,Fish Market


Creating Neighborhoods Clusters

In [49]:
kclusters = 5
manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
manhattan_merged.head(10)  

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Japanese Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,French Restaurant,Food Truck,Food & Drink Shop,Fish Market,Dessert Shop,Deli / Bodega
1,Manhattan,Chinatown,40.715618,-73.994279,3,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Korean Restaurant,Noodle House,Asian Restaurant,Café,Sake Bar,Vegetarian / Vegan Restaurant,Food & Drink Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,French Restaurant,Food Truck,Food & Drink Shop,Fish Market,Dessert Shop
3,Manhattan,Inwood,40.867684,-73.921210,4,Sushi Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Grocery Store,French Restaurant,Food Truck,Food & Drink Shop,Fish Market,Dessert Shop,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Sushi Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Noodle House,Cocktail Bar,Deli / Bodega,French Restaurant,Food Truck,Food & Drink Shop,Fish Market
5,Manhattan,Manhattanville,40.816934,-73.957385,1,Sushi Restaurant,Japanese Restaurant,Japanese Curry Restaurant,Noodle House,Ramen Restaurant,Cocktail Bar,Deli / Bodega,Food Truck,Food & Drink Shop,Fish Market
6,Manhattan,Central Harlem,40.815976,-73.943211,1,Sushi Restaurant,Japanese Restaurant,Japanese Curry Restaurant,Ramen Restaurant,Noodle House,Cocktail Bar,Deli / Bodega,Food Truck,Food & Drink Shop,Fish Market
7,Manhattan,East Harlem,40.792249,-73.944182,0,Sushi Restaurant,Japanese Restaurant,Hawaiian Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,French Restaurant,Food Truck,Food & Drink Shop,Fish Market,Dessert Shop
8,Manhattan,Upper East Side,40.775639,-73.960508,0,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Cocktail Bar,Dessert Shop,French Restaurant,Food Truck
9,Manhattan,Yorkville,40.775930,-73.947118,0,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Fish Market,Indian Chinese Restaurant,Chinese Restaurant,Café


In [51]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)    
map_clusters

In [58]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Sushi Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Noodle House,Cocktail Bar,Deli / Bodega,French Restaurant,Food Truck,Food & Drink Shop,Fish Market
7,East Harlem,Sushi Restaurant,Japanese Restaurant,Hawaiian Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,French Restaurant,Food Truck,Food & Drink Shop,Fish Market,Dessert Shop
8,Upper East Side,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Cocktail Bar,Dessert Shop,French Restaurant,Food Truck
9,Yorkville,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Fish Market,Indian Chinese Restaurant,Chinese Restaurant,Café
10,Lenox Hill,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Fish Market,Cocktail Bar,Beer Bar,Chinese Restaurant
11,Roosevelt Island,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Ramen Restaurant,Cocktail Bar,Beer Bar,Seafood Restaurant,Deli / Bodega,Food Truck,Food & Drink Shop
25,Manhattan Valley,Sushi Restaurant,Japanese Restaurant,Hawaiian Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,French Restaurant,Food Truck,Food & Drink Shop,Fish Market,Dessert Shop
26,Morningside Heights,Sushi Restaurant,Japanese Restaurant,Japanese Curry Restaurant,Ramen Restaurant,Hawaiian Restaurant,Deli / Bodega,French Restaurant,Food Truck,Food & Drink Shop,Fish Market
30,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Building,Café,Chinese Restaurant,Cocktail Bar


In [54]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,French Restaurant,Food Truck,Food & Drink Shop,Fish Market,Dessert Shop
5,Manhattanville,Sushi Restaurant,Japanese Restaurant,Japanese Curry Restaurant,Noodle House,Ramen Restaurant,Cocktail Bar,Deli / Bodega,Food Truck,Food & Drink Shop,Fish Market
6,Central Harlem,Sushi Restaurant,Japanese Restaurant,Japanese Curry Restaurant,Ramen Restaurant,Noodle House,Cocktail Bar,Deli / Bodega,Food Truck,Food & Drink Shop,Fish Market
12,Upper West Side,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Japanese Curry Restaurant,Asian Restaurant,Grocery Store,Sake Bar,Deli / Bodega,Food Truck,Food & Drink Shop
13,Lincoln Square,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Japanese Curry Restaurant,Smoothie Shop,Noodle House,Chinese Restaurant,Sake Bar,Soba Restaurant,Asian Restaurant
17,Chelsea,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Japanese Curry Restaurant,Asian Restaurant,Udon Restaurant,Fish Market,Cocktail Bar,Deli / Bodega,Dessert Shop
18,Greenwich Village,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Udon Restaurant,Japanese Curry Restaurant,Coffee Shop,Dessert Shop,Noodle House,Cocktail Bar,Soba Restaurant
21,Tribeca,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Japanese Curry Restaurant,Noodle House,Asian Restaurant,Udon Restaurant,Korean Restaurant,New American Restaurant,Dessert Shop
22,Little Italy,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Korean Restaurant,Noodle House,Japanese Curry Restaurant,Udon Restaurant,Sake Bar,Café,Asian Restaurant
23,Soho,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Noodle House,Japanese Curry Restaurant,Udon Restaurant,Korean Restaurant,New American Restaurant,Dessert Shop,Food & Drink Shop


In [55]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Japanese Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,French Restaurant,Food Truck,Food & Drink Shop,Fish Market,Dessert Shop,Deli / Bodega


In [56]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Korean Restaurant,Noodle House,Asian Restaurant,Café,Sake Bar,Vegetarian / Vegan Restaurant,Food & Drink Shop
14,Clinton,Sushi Restaurant,Ramen Restaurant,Japanese Restaurant,Japanese Curry Restaurant,Seafood Restaurant,Asian Restaurant,Chinese Restaurant,Cocktail Bar,Restaurant,American Restaurant
15,Midtown,Japanese Restaurant,Ramen Restaurant,Sushi Restaurant,Asian Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Sandwich Place,Food & Drink Shop,Restaurant,French Restaurant
16,Murray Hill,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Tonkatsu Restaurant,Soba Restaurant,French Restaurant,Restaurant
19,East Village,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Sake Bar,Café,Dessert Shop,Udon Restaurant,Coffee Shop,Japanese Curry Restaurant,Shabu-Shabu Restaurant
20,Lower East Side,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Sake Bar,Café,Dessert Shop,Udon Restaurant,Korean Restaurant,Coffee Shop,Japanese Curry Restaurant
27,Gramercy,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Udon Restaurant,Soba Restaurant,Japanese Curry Restaurant,Dessert Shop,Coffee Shop,Sake Bar
31,Noho,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Japanese Curry Restaurant,Café,Dessert Shop,Udon Restaurant,Coffee Shop,Noodle House,Sake Bar
33,Midtown South,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Japanese Curry Restaurant,Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Shabu-Shabu Restaurant,Food Truck
34,Sutton Place,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Japanese Curry Restaurant,Seafood Restaurant,Asian Restaurant,Food & Drink Shop,Deli / Bodega,French Restaurant,Taco Place


In [57]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Sushi Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Grocery Store,French Restaurant,Food Truck,Food & Drink Shop,Fish Market,Dessert Shop,Deli / Bodega
